In [ ]:
import numpy
import numpy.ma as ma
import scipy.signal
import shapely
from shapely.geometry import Polygon as PPGON
from rasterio.plot import show
from rasterio.merge import merge
from rasterio.fill import fillnodata
from rasterio.mask import mask
import rasterio
from osgeo import gdal
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em
from skimage import io, data, img_as_float
from skimage import exposure
from rasterio.plot import show_hist
import geopandas as gpd
import subprocess as sp
import xrspatial
import glob
import osmnx as ox
import zipfile
from sklearn.cluster import KMeans
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
ox.config(use_cache=True, log_console=True)
import ee
import geemap

In [ ]:
service_account = 'pandas@dev-ind-geo-01.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/workspace/snapdocker/credentials.json')
ee.Initialize(credentials=credentials)

In [ ]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    '/workspace/snapdocker/credentials.json',
)

In [ ]:
litto = gpd.read_file('littoral.geojson')
litto.plot()

In [ ]:
aoi = geemap.geopandas_to_ee(litto).geometry()

In [ ]:
region=aoi.getInfo()

In [ ]:
start_date = '2021-06-01'
end_date = '2021-06-30'

In [ ]:
# Function to mask clouds using the Sentinel-2 QA band

def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)

In [ ]:
se2 = (ee.ImageCollection('COPERNICUS/S2')
        .filterDate(start_date,end_date)
        .filterBounds(aoi)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",70))
        .map(se2mask)
        .select(['B2','B3','B4','B8'])
        .median())

In [ ]:
# Create Dynamic World land cover composite
# landcover = geemap.dynamic_world(aoi, start_date, end_date, return_type='hillshade')

In [ ]:
rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}


# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(se2.clip(aoi), rgbViz, "S2")
# map1.addLayer(landcover.clip(aoi), {}, 'Land Cover')
region_im = ee.Image().byte().paint(
    featureCollection=ee.Geometry(region), width=2, color=1
)
map1.addLayer(region_im, dict(palette=['FF0000']), 'Region')
map1.addLayerControl()
map1

In [ ]:
# capture_date = '20210430'
capture_date = '20210630'

In [ ]:
outputBucket = 'geotiff01' #Change for your Cloud Storage bucket

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toCloudStorage(**{
    'image': se2.clip(aoi),
    'description': capture_date+'_',
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'bucket': outputBucket,
    'formatOptions': {'cloudOptimized': False},
    'path':'cameroondata/raw/'+capture_date+'/',
    'scale':10,
    'maxPixels':int(1e13)
})

task.start()
import time 
count = 1
while task.active():
    print(f'{count}' + ': Polling for task (id: {}).'.format(task.id))
    time.sleep(600)
    count +=1

In [ ]:
# gee_mosaic_list = [
#     '/workspace/sentinelproduct/cameroon/02103310000022016-0000000000.tif',
#     '/workspace/sentinelproduct/cameroon/202103310000000000-0000000000.tif',
#     '/workspace/sentinelproduct/cameroon/202103310000000000-0000011008.tif',
#     '/workspace/sentinelproduct/cameroon/202103310000011008-0000000000.tif',
#     '/workspace/sentinelproduct/cameroon/202103310000011008-0000011008.tif',
#     '/workspace/sentinelproduct/cameroon/202103310000022016-0000011008.tif',
# ]

In [ ]:
# for i in range(0,6):
#     img = rasterio.open(gee_mosaic_list[i],nodata=np.nan)
#     show(img)
#     print(img.profile)

## RUN gsutil cp "gs://geotiff01/cameroondata/raw/20210430/*.tif" ../sentinelproduct/cameroon/20210430/

In [ ]:
import glob

gee_mosaic_list = []
for file in glob.glob("/workspace/sentinelproduct/cameroon/"+capture_date+'/*.tif'):
    gee_mosaic_list.append(file)
gee_mosaic_list

In [ ]:
# tile=[]
# for i in range(0,6):
#     src = rxr.open_rasterio(gee_mosaic_list[i],masked=True)
#     tile.append(src[0])
# merged = merge_arrays(tile)

In [ ]:
# merged.rio.to_raster("/workspace/sentinelproduct/cameroon/20210331.tif")

In [ ]:
geetr = []
for raster in gee_mosaic_list:
    out_file = raster.split('.')[0]+'_tr.tif'
    geetr.append(out_file)
    print('Creating file '+out_file+ '....')
    args = args = ['gdal_translate', '-a_nodata','nan', '-of', 'Gtiff', raster , out_file]
    result = sp.call(args)
geetr

In [ ]:
gee_mosaic_name = "/workspace/sentinelproduct/cameroon/"+capture_date+"/"+capture_date+".tif"

In [ ]:
# gee_mosaic_name = "/workspace/sentinelproduct/cameroon/20210331.tif"

g = gdal.Warp(gee_mosaic_name, geetr, format="GTiff",) # if you want
g = None # Close file and flush to disk

In [ ]:
# src=rxr.open_rasterio(gee_mosaic_name, masked=True)

In [ ]:
# src.plot()

In [ ]:
src = rasterio.open(gee_mosaic_name,masked=True)

In [ ]:
show(src.read(2))